In [ ]:
!pip install ipython==7.34.0 ipykernel==5.5.6
!pip install import_ipynb

In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import dolfin
else:
    try:
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import dolfin

import numpy as np
import matplotlib.pyplot as plt
import mshr
import fenics as fe
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import import_ipynb

path = '/content/drive/MyDrive/Colab Notebooks/Physics-Informed Neural Networks/Demo/fenics_cfd/generate_data'
os.chdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from vessel_geometry import generate_vessel_geometry
from boundaries import define_boundaries
from navier_stokes import solve_navier_stokes
from visualize_geometry import visualize
from generate_random_geometries import random_vessel_type, random_vessel_params

In [ ]:
def to_meters(size):
  return size * 10**(-3)

In [ ]:
def get_inflow_profile(height, max_velocity):
  inflow_profile = (f'4 * {max_velocity} * x[1] / {height} * (1 - x[1] / {height})', '0')
  return inflow_profile

In [ ]:
def define_function_spaces(mesh):
  V = fe.VectorFunctionSpace(mesh, 'P', 2)
  Q = fe.FunctionSpace(mesh, 'P', 1)

  return V, Q

In [ ]:
def get_optimal_time_step(mesh, max_velocity):

  Courant_number = 0.5
  smallest_cell_size = mesh.hmin()
  dt = Courant_number * smallest_cell_size / max_velocity

  return dt

In [ ]:
def get_height(mesh):

  x_coords = mesh.coordinates()[:, 0]
  y_coords = mesh.coordinates()[:, 1]
  length = x_coords.max() - x_coords.min()

  threshold = x_coords.min() + length * 0.1
  filtered_y_coords = y_coords[x_coords <= threshold]

  height = filtered_y_coords.max() - filtered_y_coords.min()

  return height

In [ ]:
def generate_random_mesh(length, height, target_resolution):

  vessel_type = random_vessel_type()
  vessel_params = random_vessel_params(vessel_type, length, height)

  print(f"Generating {vessel_type} type vessel\n")

  length, height = to_meters(length), to_meters(height)

  domain = generate_vessel_geometry(vessel_type, length, height, params=vessel_params)
  mesh = mshr.generate_mesh(domain, target_resolution)

  return mesh

In [ ]:
def generate_flow_field(mesh, mu_scalar, rho_scalar, num_steps):

  inflow_velocity = to_meters(np.random.uniform(20, 50))  # inflow velocity (milimeters per second)
  dt = get_optimal_time_step(mesh, inflow_velocity)       # time step size (seconds)
  scalars = [dt, mu_scalar, rho_scalar]

  V, Q = define_function_spaces(mesh)
  boundaries = define_boundaries(mesh)
  inflow_profile = get_inflow_profile(get_height(mesh), inflow_velocity)

  start = time.time()
  u_, p_, diverged = solve_navier_stokes(V, Q, mesh, boundaries, inflow_profile, scalars, num_steps, 1e-4, 'lu')
  end = time.time()

  print(f"Executed in {end - start :.2f} s\n")

  return u_, p_, diverged